# Demo

I am recreating the important parts of the [dm_control_suite](https://github.com/google-deepmind/mujoco_playground/blob/main/learning/notebooks/dm_control_suite.ipynb) example.

In [ ]:
%env MUJOCO_GL=egl

import os

import jax
import mediapy as media
from tqdm import tqdm
import twmr  # noqa: F401
import warp as wp
from jax import numpy as jp
from mujoco_playground import registry

wp.config.quiet = True

# Tell XLA to use Triton GEMM, this improves steps/sec by ~30% on some GPUs
xla_flags = os.environ.get("XLA_FLAGS", "")
xla_flags += " --xla_gpu_triton_gemm_any=True"
os.environ["XLA_FLAGS"] = xla_flags

env = registry.load("TransformableWheelMobileRobot")

state = env.reset(jax.random.PRNGKey(0))

action = jp.ones(env.action_size) * 0.01
# action = jp.zeros(env.action_size)
# TODO: state.data.ctrl... what is the difference?

rollout = [state]
for _ in tqdm(range(250), desc="Simulating steps"):
    state = env.step(state=state, action=action)
    rollout.append(state)

print(f"Final position: {state.data.qpos}")

# TODO: set the camera to "closeup"?
# renderer.update_scene(data, camera="closeup")

frames = env.render(rollout)
media.show_video(frames, fps=1.0 / env.dt)

In [ ]:
env_cfg = registry.get_default_config("TransformableWheelMobileRobot")